## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [5]:
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def RunPulseRateAlgorithm(data_fl, ref_fl, fs=125):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Compute pulse rate estimates and estimation confidence.
    #select bandpass filter within 40-240 Hz
    ppg, accx, accy, accz = BandFilter(ppg, fs), BandFilter(accx, fs), BandFilter(accy, fs), BandFilter(accz, fs)
    
    # 3D accelerometer average 
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # Load ground truth as column vector and convert to row vector
    ground_truth = scipy.io.loadmat(ref_fl)['BPM0'].reshape(-1) 
    
    # Calculate heart rate prediction, confidence and errors
    predictions, confidence = PredictHeartRate(acc, ppg, fs)
    errors = np.abs(np.subtract(predictions, ground_truth))
    
    # Return per-estimate mean absolute error and confidence
    errors, confidence = np.array(errors), np.array(confidence)
    return errors, confidence

def BandFilter(signal, fs, bandpass=(40/60.0, 240/60.0)):
    """ Function to filter the frequency between 20 Hz and 240 Hz.
    Args:
        signal: A numpy array of signal 
        fs (int): Sampling rate of the signal
        bandpass(tuple):bandpass within 40-240 Hz
    Returns:
        predictions, confidences
    """
    b, a = scipy.signal.butter(3, bandpass, btype='bandpass', fs=fs) # 3: Order of the filter 
    return scipy.signal.filtfilt(b, a, signal)


def FourierTransform(signal, fs):
    """
    Calculate the Fast Fourier Transfrom of given sequence
    Args:
        signal: A numpy array for ppm or acc signal
        fs (int): Sampling frequency in Hz
    Returns:
        freqs: A numpy array Frequency bins 
        fft (float): Magnitude of FFT
    """ 
    n_samples = len(signal)  # no zero padding
    freqs = np.fft.rfftfreq(n_samples, 1/fs)
    fft = np.abs(np.fft.rfft(signal, n_samples))
    return freqs, fft


def CalculateConfidence(freqs, fft, freq, window_width =1):
    """ Function to find the confidence level from Fourier transform 
    Args:
        freqs: A numpy array of frequency 
        ffts: A numpy array of FFT magnitude
        freq (float): frequency to calculate the confidence
    Returns:
        confidence
    """
    window = (freqs > freq - window_width) & (freqs < freq + window_width)
    confidence = np.sum(fft[window]) / np.sum(fft)
    
    return confidence

def PredictHeartRate(acc, ppg, fs):
    """
    Function to predict heart rate and calculate confidence using ppg and accelerometer data

    Args:
        acc: A numpy array of acceleration signal file
        ppg: A numpy array of PPG signal file
        fs (int): Sampling frequency in Hz
    Returns:
        predictions: A list of predicited heart rates
        confidence: A list of estimated confidences
    """
    # Define window shapes with given values to function
    window_length=8
    window_shift=2
    min_bandpass=40/60.0 
    max_bandpass=240/60.0
    window_length = window_length * fs
    window_shift = window_shift * fs
    
    # Create empty lists to store calculations
    bpm_predictions = []
    confidence = []
    
    # Make calculations with shifting windows
    for i in range(0, len(ppg) - window_length + 1, window_shift):
        
        # Estimation for current window
        ppg_window = ppg[i: i + window_length]
        acc_window = acc[i: i + window_length]
        
        # Fourier transform
        ppg_freqs, ppg_fft = FourierTransform(ppg_window, fs)
        acc_freqs, acc_fft = FourierTransform(acc_window, fs)
        
        # Remove unwanted frequencies 
        ppg_fft[ppg_freqs <= min_bandpass] = 0.0
        ppg_fft[ppg_freqs >= max_bandpass] = 0.0
        acc_fft[acc_freqs <= min_bandpass] = 0.0
        acc_fft[acc_freqs >= max_bandpass] = 0.0
    
        # Pick frequency with largerst FFT coefficient
        ppg_freq = ppg_freqs[np.argmax(ppg_fft, axis=0)]  
        acc_freq = acc_freqs[np.argmax(acc_fft, axis=0)]
        
        freq = ppg_freq
        conf = CalculateConfidence(ppg_freqs, ppg_fft, ppg_freq)
        
        # If the dominant accelerometer frequency is the same as the PPG, estimated heart rate is biased.
        # Make new estimation sor second suitable frequency.
        if abs(ppg_freq - acc_freq) == 0:
            
            # Calculate frequency with second largerst FFT
            sec_freq = ppg_freqs[np.argsort(ppg_fft, axis=0)[-2]]
            sec_conf = CalculateConfidence(ppg_freqs, ppg_fft, sec_freq)
            
            # If new freq is larger, choose this as freq
            if sec_conf > conf:
                freq, conf = sec_freq, sec_conf
        
        bpm_predictions.append(freq * 60) # Convert to bpm (from Hz)
        confidence.append(conf)
        
    return bpm_predictions, confidence


In [6]:
Evaluate()

15.726958002724126

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

## Code Description 

The dependencies packages required to run the algorithm can be downloaded through `pip freeze`. Once the requirement packages have been installed and notebook file to the algorithm is saved. You can run the code block cell containing all the defined functions. Then run the `Evaluate()` to give the **MEAN ABSOLUTE ERROR**.

## Data Description

This is a Troika datasets from a journal paper; Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. 

Two-channel PPG signals, three-axis acceleration signals, and one-channel ECG signals were simultaneously recorded from subjects with age from 18 to 35. There are 12 subjects and they are all males. For each subject, the PPG signals were recorded from wrist by two pulse oximeters with green LEDs (wavelength: 515nm). All signals were sampled at 125 Hz and sent to a nearby computer via Bluetooth.
Each dataset contains a variable 'sig'. It has 6 rows. The first row is a simultaneous recording of ECG, which is recorded from the chest of each subject. The second row and the third row are two channels of PPG, which are recorded from the wrist of each subject. The last three rows are simultaneous recordings of acceleration data (in x-, y-, and z-axis).
The short-coming with the dataset is that is the demographic representative,it does not include females candidate. The age could also have an impact because people below 18 and above 35 most likely to use the algorithm might not give a better results since the model was not exposed to similar data. 

## Algorithm Description

The algorithm takes into account the heart rate and how confident it is. Firstly, we limit our sampling rate between 40bpm to 240bpm. Since the subjects could be moving their arm due to exercise program, ppg sensor might pick signals which ain't from the heart rhythm. The algorithm deals with this by picking frequency that has largest dominant and search for peaks within the range of the dominating frequency to calculate how confident it is. The algorithm compares the ground truth to the predicted freqency (how distant is the predicted values are to the ground truth). The algorithm picks the 90% strongest confidence values and find the average of their corresponding absolute error.

The algorithm takes advantage of the blood capillaries and the green light from the PPG sensor emits into the vessels. When the heart ventricles contract, it pumps bloods to the body capillaries and this increases the volume of blood vessel thus, this absorbes the green lights, little light will be reflected and detected by the photodector. Once the heart ventricles repolarised, the bloods leave the capillaries and more light is reflected back tot the photodector.

The model assumes the wearer is performing intense activity and it tries to carefully calculate it's metric by removing the arm movement artifacts. Occasionally, it might pick the arm movements and gives false outputs resulting in failure mode.

The training datasets gives mean absolute error(MAE) of **15.7** and the test dataset gives **9.13**
The lower the MAE, the better the perforomance of the algorithm. This implies their still improvement to be consider for the algorithm on the training dataset. Maybe selecting a model train_test_split for the train and test will improve the performance of the model. There will be limition since the dataset are collected from exercise programs that include only, running, jogging and walking. There would be performance derange on users under the water swimming and  demographic limitation of only male and age bin 18-35 could give population outside of this reference a failure modes. 

In [2]:
!pip freeze

altair==1.2.1
asn1crypto==0.22.0
atari-py==0.1.7
atomicwrites==1.3.0
attrs==19.1.0
audioread==2.1.6
av==0.3.3
awscli==1.16.17
backcall==0.1.0
backports.functools-lru-cache==1.4
backports.weakref==1.0rc1
beautifulsoup4==4.6.0
bleach==1.5.0
blinker==1.4
bokeh==0.12.13
boto==2.48.0
boto3==1.9.7
botocore==1.12.7
Box2D==2.3.2
Box2D-kengz==2.3.3
bresenham==0.2
bz2file==0.98
cassandra-driver==3.11.0
certifi==2023.5.7
cffi==1.11.2
chardet==3.0.4
click==6.7
cloudpickle==0.2.2
colorama==0.3.9
colour==0.1.5
conda==4.6.14
cryptography==2.1.4
cycler==0.10.0
Cython==0.29.7
cytoolz==0.9.0.1
dask==0.16.1
decorator==4.0.11
defusedxml==0.5.0
dill==0.2.7.1
docutils==0.14
dronekit==2.9.2
entrypoints==0.2.3
et-xmlfile==1.0.1
eventlet==0.22.0
fastcache==1.0.2
Flask==0.12.2
Flask-SocketIO==2.9.2
future==0.16.0
gensim==3.4.0
glfw==1.8.0
greenlet==0.4.12
gym==0.9.6
h5py==2.7.1
heapdict==1.0.0
html2text==2018.1.9
html5lib==0.9999999
idna==2.6
imageio==2.1.2
ipykernel==4.9.0
ipython==6.5.0
ipython-genutils==0.2.

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 